In [1]:
import pandas as pd
import numpy as np

In [2]:
dailydf = pd.read_csv('Daily-tushare-20160101-20190430.csv')
dailydf = dailydf.drop('Unnamed: 0', axis=1, inplace=False)
indexdf = pd.read_csv('Index-tushare-20160101-20190430.csv')
indexdf = indexdf.drop('Unnamed: 0', axis=1, inplace=False)
daterange = indexdf.drop_duplicates(subset=['trade_date'], keep='first')['trade_date']

In [3]:
def decideRange(date):
    dr = daterange.values.tolist()
    while dr[-1] > date:
        dr.pop()
    return dr[-1]

In [4]:
indexdailydf = dailydf.drop_duplicates(subset=['trade_date'], keep='first')
indexdailydf = indexdailydf.drop('ts_code', axis=1, inplace=False)
indexdailydf = indexdailydf.drop('open', axis=1, inplace=False)
indexdailydf = indexdailydf.drop('high', axis=1, inplace=False)
indexdailydf = indexdailydf.drop('low', axis=1, inplace=False)
indexdailydf = indexdailydf.drop('close', axis=1, inplace=False)

In [5]:
indexnlist = []
for date in indexdailydf['trade_date']:
    weights = indexdf[indexdf['trade_date']==decideRange(date)]
    weightlist, pricelist, indexn = weights['weight'].tolist(), [], 0
    for con in weights['con_code']:
        prices = dailydf[dailydf['ts_code']==con]
        try:
            pricelist.append(prices[prices['trade_date']==date]['close'].values[0])
        except:
            pricelist.append(0)
    indexn = np.multiply(np.array(weightlist)/100, np.array(pricelist)).sum()
    indexnlist.append(indexn)

KeyboardInterrupt: 

In [ ]:
indexdailydf.insert(1, 'value', indexnlist)
indexdailydf

In [ ]:
changelist = []
for i in range(0, len(indexnlist)):
    if i != 0:
        changelist.append((indexnlist[i]-indexnlist[i-1])/indexnlist[i-1])
    else:
        changelist.append(0)
indexdailydf.insert(2, 'change', changelist)
indexdailydf

In [8]:
indexdailydf.to_csv('Index-Daily-20160101-20190430.csv')